In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#libraries 
#%matplotlib notebook
import plotly.express as px
import pandas as pd
import numpy as np
import warnings
import itertools
import seaborn
import matplotlib.dates as md
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')

import statsmodels.api as sm 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest


LECTURA DE DATOS


In [3]:
URL = 'https://drive.google.com/file/d/1ImirryRX-CqFYhQxtNO7s3NLMirfgdlx/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+URL.split('/')[-2]

df = pd.read_csv(path,delimiter=';',parse_dates=['DateObserved'])
#filename = ''
#df = pd.read_csv(filename, delimiter = ',', parse_dates=['DateObserved'])
df.head(10)

,Id,Machine,DateStr,DateObserved,H,V,C
0,921880,Batidora,01/09/2018 13:39:16,2018-09-01 13:39:16,0.18,0.14,0.1
1,921881,Batidora,01/09/2018 13:39:46,2018-09-01 13:39:46,0.17,0.15,0.1
2,921882,Batidora,01/09/2018 13:40:15,2018-09-01 13:40:15,0.17,0.17,0.1
3,921883,Batidora,01/09/2018 13:42:56,2018-09-01 13:42:56,0.17,0.16,0.1
4,921884,Batidora,01/09/2018 13:43:26,2018-09-01 13:43:26,0.17,0.19,0.1
5,921885,Batidora,01/09/2018 13:43:56,2018-09-01 13:43:56,0.20,0.18,0.1
6,921886,Batidora,01/09/2018 13:44:26,2018-09-01 13:44:26,0.18,0.17,0.1
7,921887,Batidora,01/09/2018 13:44:55,2018-09-01 13:44:55,0.18,0.17,0.1
8,921888,Batidora,01/09/2018 13:45:25,2018-09-01 13:45:25,0.22,0.16,0.1
9,921889,Batidora,01/09/2018 13:45:55,2018-09-01 13:45:55,0.18,0.16,0.1


Selecciono los datos de la batidora y elimino las columnas inncecesarias

In [4]:
batidora = df.loc[df['Machine'] == 'Batidora']
cols = ['Id', 'DateStr', 'Machine']
batidora.drop(cols, axis=1, inplace=True)
batidora = batidora.set_index('DateObserved')

Compruebo los datos que son nulos

In [5]:
batidora.isnull().sum()

H    28
V     0
C     7
dtype: int64

Agrupo el dataset en grupos de 15 minutos donde aplico la media Después de agrupar pongo los nulos a la media

In [6]:
batidora_resample15m = batidora.resample('15min').mean()
batidora_resample15m['H'].fillna(batidora_resample15m['H'].min(),inplace=True)
batidora_resample15m['C'].fillna(batidora_resample15m['C'].min(),inplace=True)
batidora_resample15m['V'].fillna(batidora_resample15m['V'].min(),inplace=True)

batidora_resample15m.head()

,H,V,C
DateObserved,,,
2018-09-01 13:30:00,0.177500,0.166250,0.100000
2018-09-01 13:45:00,0.181667,0.169000,0.100000
2018-09-01 14:00:00,0.183871,0.166452,0.100323
2018-09-01 14:15:00,0.190323,0.170968,0.102581
2018-09-01 14:30:00,0.195806,0.183871,0.100000


# Isolation Forest

In [7]:
#Take useful feature and standardize them
#determinamos el outlaier en est caso 1 %
outliers_fraction=0.01
#extraemos los datos, donde en este caso son importante las 3
data = batidora_resample15m[['H','V','C']]
#entrenamos un arbol con 100 arboles sin que sea paralelo con un random state igual a 42
#train isolation forest 
model = IsolationForest(contamination = outliers_fraction, n_estimators=1000, max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42,verbose=1)
model.fit(data)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.9s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    2.9s finished


IsolationForest(contamination=0.01, n_estimators=1000, n_jobs=-1,
                random_state=42, verbose=1)

#Guardo el modelo

In [8]:
#esto me genera un archivo que se llama 'isolation_forest.pickle'
import pickle
filename = 'isolation_forest.pickle'
pickle.dump(model, open(filename, 'wb'))

In [11]:
#genero una funcion para predecir anomalias pasandole el modelo y los datos, en nuestro caso
#sera un registro que tenga la fecha direccion vertical horizontal y cprriente
#creamos un nuevo dataframe por que la prediccion requiere de un dataframe 
import pandas as pdnew
def predecir(model, datos):
  #nuevo_dato={'DateObserved':[now], 'H':[0.2], 'V':[0.2], 'C':[0.11]}
  df = pdnew.Dataframe(datos)
  df = df.set_index('DateObserved')
  #realizo prediccion y calculo distancia
  iso_prediction = model.predict(df)
  iso_core = model.score_samples(df) #mide la distancia
  #calculo si es una anomalia y su probabilidad
  anomaly =(iso_prediction[0]==-1)# si eso es igul a -1 es una anomalia
  probabilty = iso_core[0] *100*-1*anomaly #si es anomalia que saque la probabbilidad 
  f'anomaly {anomaly} probabilidad {probabilty}'
  return anomaly, probabilty

In [12]:
#creo un dato
from datetime import datetime 
from datetime import date 
now = datetime.now()
today = date.today()
nuevo_dato = {'DateObserved':[now], 'H':[0.2], 'V':[0.2], 'C':[0.11]}
predecir(model, nuevo_dato)


AttributeError: ignored

In [ ]:
filename = 'isolation_forest.pickle'
model1 = pickle.load(open(filename, 'rb'))
predecir(model1, nuevo_dato)